In [ ]:
# Multiparameter Sweep Code 

""" $ import libraries $"""
import os
import numpy as np
import pandas as pd
from itertools import product
from Multiparameter_Sweep_Functions import * 


"""$ define params $ """

# Set for Temperature (°C) 

Temp_low, Temp_high, Temp_interval = 20.0, 100.0, 10 

# generate random gauss distribution between high low endmembers
Temp_values = np.linspace(Temp_low, Temp_high, num=int((Temp_high - Temp_low) / Temp_interval) + 1, endpoint=True)


# Set for NaOH Concentration (mM) 

NaOH_low, NaOH_high = 32.0, 1000.0 

# generate random gauss distribution between high low endmembers
NaOH_values = np.array([NaOH_low, 50.0, 100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 800.0, 900.0, NaOH_high])

# Set for Mg++ Concentration (mM)
Mg_low, Mg_high = 32.0, 660.0  # Example range for Mg++

# Generate random Gaussian distribution for Mg++ Concentration
Mg_values = np.array([Mg_low, 80.0, 160.0, 250.0, 300.0, 350.0, 400.0, 450.0, 550.0, Mg_high])

# Set for Seed Volume Fraction (m^3 mineral/m^3 medium)
Seed_low, Seed_high = 1.0*10**-10, 0.0032  # Example range for Seed Volume

# Generate random Gaussian distribution for Seed Volume
Seed_values = np.array([Seed_low, 1*10**-9, 1*10**-8, 1*10**-7, 1*10**-6, 1*10**-5, 1*10**-4, Seed_high])


#>> for multiple parameters uncomment the line below and modify..
# var_dic={'Temperature': Temp_values.copy(), 
#          'NaOH': NaOH_values.copy(),
#          'Mg++': Mg_values.copy(), 
#          'Seed Volume Fraction': Seed_values.copy()}

# # Find the length of the longest array
# max_length = max(len(arr) for arr in var_dic.values())

# # Pad shorter arrays with NaN to match the length of the longest array so that I can convert into dataframe 
# for key in var_dic:
#     if len(var_dic[key]) < max_length:
#         var_dic[key] = np.pad(var_dic[key], (0, max_length - len(var_dic[key])), constant_values=np.nan)

# #>> export to excel sheet
# var_df = pd.DataFrame(var_dic)
# var_df.to_excel('parameter_sensitivity_ic_variable_values.xlsx')


"""$ import values into input file and run crunch $ """

# *** you will need to specify path to your crunch executable to run
# properly!!! ***
crunch_exe_path = '/Users/tszkipeterwei/Documents/crunchtope/M1MacPre-compiled/CrunchTope-Mac'

# Generate all combinations of the parameters 
param_combinations = np.array(list(product(Temp_values, NaOH_values, Mg_values, Seed_values)))

print("Generated parameter combinations")
print(param_combinations)

In [ ]:
# Determine the Length of Parameter Combinations
print(len(param_combinations))

In [ ]:
# Test the Algorithm for the Multiparameter Sweep (Hopefully does not take forever)


n = 1 # counter of simulations 
data = [] # dataset with all the simulations
tseries = {}  # Creates an empty dictionary for holding the time series 
for param_set in param_combinations:
    '''update ic variable values each iteration through'''

    #print(param_set)

    # Alters the Temperature and the Mineral Reaction Kinetics in InitialStage.in 
    Temp(param_set[0])

    # Alters the NaOH, Mg, and Seed Volume Fraction, Runs the input files, outputs the combined time series 
    Model_Run = Time_Series('PestControl.ant', param_set[1], param_set[2], param_set[3])

    # If CrunchTope ran properly 
    if Model_Run is not None: 
        # Output the carbonation percentage (%) and reaction rate (mol/[kgw *hr])
        Reaction_Rate, Carbonation_Percentage = Magnesium_Rate(Model_Run, param_set[3])

        ''' model run output storage'''

        entry = {
            'Label': f'Run_{n}',  # Unique identifier for each run
            'Temperature [°C]': param_set[0],
            'NaOH Concentration [mM]': param_set[1],
            'Mg++ Concentration [mM]': param_set[2],
            'Seed Volume Fraction [m^3 mineral/m^3 medium]': param_set[3],
            'Carbonation_Percentage': Carbonation_Percentage,
            'Reaction_Rate': Reaction_Rate
        }

        data.append(entry)

        # Store the Time Series into a Dictionary 
        tseries['sim'+str(n)] = Model_Run 
        print('sim'+str(n)) 
        n = n + 1 
    # If CrunchTope was terminated 
    else:
        entry = {
            'Label': f'Run_{n}',  # Unique identifier for each run
            'Temperature [°C]': param_set[0],
            'NaOH Concentration [mM]': param_set[1],
            'Mg++ Concentration [mM]': param_set[2],
            'Seed Volume Fraction [m^3 mineral/m^3 medium]': param_set[3],
            'Carbonation_Percentage': 'NaN',
            'Reaction_Rate': 'NaN'
        }

        data.append(entry)

        # Store the Time Series into a Dictionary 
        tseries['sim'+str(n)] = "Failed Run" 
        print('Skipped sim'+str(n)) 
        n = n + 1 

print(data)
print(tseries)
print('Finished')

# Save the file now

In [ ]:
# Post-Data Processing 

# Turn array into dataframe 
df = pd.DataFrame(data)

# Export to CSV File (Large Dataset, .xlsx file might take too long to run)
df.to_csv('Multiparameter_Sweep_Runs.csv', index=False)

print(df) 

# Save the file now 

In [ ]:
'''*** OPTIONAL EXPORT TO LOCAL DIRECTORY *****'''
# user-input
export_option = 'yes'  # user option = 'no' or 'yes'

# this code is only exporting timeseries batch output to files in local directory

# create output folder if it doesn't already exist
if not os.path.exists('output'):
    os.mkdir('output')

# loop through the parameter combinations
for i in range(len(param_combinations)):
    if export_option == 'yes':
        output_fname = 'model' + str(i + 1)
        output_path = os.path.join('output', output_fname)
        
        # create directory for each simulation's output
        if not os.path.exists(output_path):
            os.mkdir(output_path)
        
        # check if the time series data exists and is not a failed run
        sim_data = tseries.get('sim' + str(i + 1))
        if isinstance(sim_data, pd.DataFrame):  # Ensure it's a DataFrame
            csv_path = os.path.join(output_path, f'batch_timeseries_sim{i + 1}.csv')
            sim_data.to_csv(csv_path, index=False)
            print(f"Saved: {csv_path}")
        else:
            print(f"Run {i + 1} failed. No data to save.")
